Cerner-HealtheIntent
HealtheIntent open development services allow access to population health concepts using RESTful APIs.

HealtheIntent Documentation

This repository contains a Python wrapper for a number of Cerner Open Code APIs

Cerner Data Syndication API - https://docs.healtheintent.com/api/v1/data_syndication/
Organizations API - https://docs.healtheintent.com/api/v1/organization/
Personnel API - https://docs.healtheintent.com/api/v1/personnel/
Subclass Definitions
Feeds aka Data Syndication
Feeds : The set of data that you want to receive on a schedule, for example, Longitudinal Record for population ABC.

Channels : Specifies that you want to receive a feed’s bundles using a particular channel type and provides any necessary configuration for the channel type. For example, a channel could specify that you want to receive your Longitudinal Record for Population ABC feed using the DOWNLOAD channel type.

Bundles : The output of a feed, for example, Changes to the Longitudinal Record for population ABC between 2018-04-01 and 2018-04-02.

Deliveries : The status of delivering a particular bundle using a particular channel. For example, you would use the Deliveries endpoint to determine whether the archive file for changes to the Longitudinal Record for population ABC between 2018-04-01 and 2018-04-02 bundle is available for you to download.

Organization
Location : A location represents a physical place with a single physical address, for example, a laboratory facility, practice site, hospital, or even a department within a hospital associated with an organization. Currently, personnel organizations and locations are not related in the API, but they will be in the future.

Organization : A formally or informally recognized group of people or organizations formed for the purpose of achieving a collective action, for example, companies, institutions, corporations, departments, community groups, and health care practice groups.

Organization Group : Organization groups are sets of organizations that are grouped for a common purpose. For example, organizations can be grouped to apply a consistent scoring methodology across organizations of the same type, filter a report to a certain set of organizations, or display a group of organizations together in applications.

Personnel
Personnel : Personnel are the people involved in the health care of a population. In most cases, personnel are employees or affiliates of a health care system or provider. : Users are personnel who log in to HealtheIntent applications. All users must be personnel, but not all personnel are necessarily users. Users gain access to features and information in HealtheIntent applications through membership in personnel groups and by being a member or administrator of organizations.

Personnel and Organization Relationship : A personnel can be a member or administrator of an organization, and some personnel may be both members and administrators of the same organization. Organization members typically are personnel who provide health care services to patients and whose financial performance and quality metrics contribute to the overall performance of the organization. Administrators are personnel who monitor and manage the activities and performance of their organizations.

Personnel Group : Personnel groups are sets of personnel who are grouped together to give them access to the same features or information. A personnel can belong to as many or as few personnel groups as necessary.

URL Structure
https://{tenant}.api.{region}.healtheintent.com/{api}/{version}/{resource}

Prerequisites
A settings.py file is required for this wrapper to work. This file should contain the variable BEARER_HEADER as the Authorization Bearer header.

Installation
!pip install Cerner-HealtheIntent
Import
import healthetintentAPI as h
Usage
o = h.Organization()
print(o.get_organization())
p = h.Personnel()
print(p.get_personnel())

In [2]:
from setuptools import setup

with open("README.md", "r") as readme_file:
    readme = readme_file.read()

setup(
    name="Cerner-HealtheIntent",
    version="0.0.1",
    author="Patrick Gagnon",
    auther_email="plgagnon00@gmail.com",
    description="A wrapper following HealtheIntent open development services which allow access to population health concepts using RESTful APIs.",
    long_description=readme,
    url="https://github.com/patrickgagnon/Cerner-HealtheIntent",
    packages=['healtheintentAPI'],
)


FileNotFoundError: [Errno 2] No such file or directory: 'README.md'

In [ ]:
class Healtheintent(object):

    def __init__(self):

        self.uri = "https://wny.api.us.healtheintent.com/"
        self.headers = BEARER_HEADER
        self.headers_op = BEARER_HEADER_OP_API
        self.channel_id = None
        self.feed_id = None
        self.personnel_id = None
        self.organization_id = None
        self.personnel_group = None
        self.personnel_group_member_id = None
        self.location_id = None
        self.body = None

    def __send_request__(self, endpoint,
                         method = None, json = None, headers = None, params = None, data = None):
        if method == 'GET':
            return requests.get('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                , headers=headers
                                , params=params).json()
        elif method == 'DELETE':
            return requests.delete('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                   , headers=headers, params=params)
        elif method == 'POST':
            return requests.post('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint),
                                   headers=headers, data=data, json=json)
        elif method == 'PUT':
            return requests.put('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint),
                                   headers=headers, params=params,data=data,json=json)
        else:
            return requests.get('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                , headers=headers, data=data, json=json)

#Data Syndication SubClass      
class DataSyndication(Healtheintent):

    def get_feeds(self):
        return self.__send_request__(endpoint='data-syndication/v1/feeds', method='GET', headers=self.headers)

    def get_channels(self):
        return self.__send_request__(endpoint='data-syndication/v1/channels', method='GET', headers=self.headers)

    def get_channel_status(self, channel_id = None):
        return self.__send_request__(endpoint='data-syndication/v1/channels/{channel_id}'.format(channel_id=channel_id),
                                     method='GET',
                                     headers=self.headers)

    def get_channel_deliveries(self, channel_id = None):
        return self.__send_request__(endpoint='data-syndication/v1/channels/{channel_id}/deliveries'.format(channel_id=channel_id),
                                     method='GET', headers=self.headers)

    def get_channel_downloads(self, delivery_id = None, destination_path = None):
        response = self.__send_request__(endpoint='data-syndication/v1/downloads/{delivery_id}'.format(delivery_id=delivery_id)
                                         , headers=self.headers)


        wd = os.getcwd()
        path = destination_path + '/' + dt.datetime.strftime(dt.datetime.today(), '%Y-%m-%d') + '.tar.gz'

        if response.status_code == 200:
            with open(path, 'wb') as i:
                i.write(response.content)
        i.close()

        temp = tarfile.open(name=path, mode='r:gz')
        os.chdir(destination_path)
        temp.extractall()
        temp.close()
        os.chdir(wd)


# Personnel Sub Class
class Personnel(Healtheintent):
    def get_personnel(self):
        return self.__send_request__(endpoint='personnel/v1/personnel', method='GET'
                                     , headers=self.headers_op)

    def create_personnel(self,data=None,json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel'.format(data=data,json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)

    def get_personnel_status(self, personnel_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_personnel(self, personnel_id=None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id, data=data, json=json)
                                     , method='PUT', headers=self.headers_op, data=data, json=json)

    def delete_personnel(self, personnel_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)

    
# Personnel Group Object
    def get_personnel_groups(self):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups'
                                     , method='GET'
                                     , headers=self.headers_op)

    def create_personnel_group(self, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups'.format(data=data,json=json)
                                     , method='POST'
                                     , headers=self.headers_op)

    def get_personnel_group_status(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_personnel_group(self, personnel_group_id = None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id, data=data,json=json)
            , method='PUT'
            , headers=self.headers_op
            , data=data
            , json=json)

    def delete_personnel_group(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)

    def get_personnel_group_members(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}/members'.format(personnel_group_id=personnel_group_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    # Testing Required
    def add_personnel_group_members(self, personnel_group_id = None, personnel_group_member_id = None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}/members/{personnel_group_member_id}'
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)

    def delete_personnel_group_members(self, personnel_group_id = None, personnel_group_member_id = None):
        return self.__send_request__(endpoint='personnel/v1/{personnel_group_id}/members/{personnel_group_member_id}'
                                     , method='DELETE'
                                     , headers=self.headers_op)


# Organization Sub Class Commands
class Organization(Healtheintent):
    
    def get_organization(self,params=None):
        return self.__send_request__(endpoint='organization/v1/organizations'.format(params=params)
                                     , method='GET'
                                     , headers=self.headers_op)
    
    def create_organization(self,data=None,json=None):
        return self.__send_request__(endpoint='organization/v1/organizations'.format(data=data, json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)
    
    def delete_organization(self, organization_id = None):
        return self.__send_request__(endpoint='organization/v1/organizations/{organization_id}'.format(organization_id=organization_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)
    
    def update_organization(self, organization_id = None, data=None, json=None):
        return self.__send_request__(endpoint='organization/v1/organizations/{organization_id}'.format(organization_id=organization_id,data=data,json=json)
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)
    
    
    # Location Object Commands
    def get_locations(self):
        return self.__send_request__(endpoint='organization/v1/locations'
                                     , method='GET'
                                     , headers=self.headers_op)

    def create_location(self,data=None,json=None):
        return self.__send_request__(endpoint='organization/v1/locations'.format(data=data, json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)
    
    def get_location_status(self, location_id = None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_location(self, location_id = None, data=None, json= None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id,data=data,json=json)
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)

    def delete_location(self, location_id = None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)

In [ ]:
from settings import *
import requests
import tarfile
import datetime as dt
import os


In [ ]:
from settings import *
import requests
import tarfile
import datetime as dt
import os

class Healtheintent(object):

    def __init__(self):

        self.uri = "https://wny.api.us.healtheintent.com/"
        self.headers = BEARER_HEADER
        self.headers_op = BEARER_HEADER_OP_API
        self.channel_id = None
        self.feed_id = None
        self.personnel_id = None
        self.organization_id = None
        self.personnel_group = None
        self.personnel_group_member_id = None
        self.location_id = None
        self.body = None

    def __send_request__(self, endpoint,
                         method = None, json = None, headers = None, params = None, data = None):
        if method == 'GET':
            return requests.get('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                , headers=headers
                                , params=params).json()
        elif method == 'DELETE':
            return requests.delete('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                   , headers=headers, params=params)
        elif method == 'POST':
            return requests.post('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint),
                                   headers=headers, data=data, json=json)
        elif method == 'PUT':
            return requests.put('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint),
                                   headers=headers, params=params,data=data,json=json)
        else:
            return requests.get('{uri}{endpoint}'.format(uri=self.uri, endpoint=endpoint)
                                , headers=headers, data=data, json=json)

#Data Syndication SubClass      
class DataSyndication(Healtheintent):

    def get_feeds(self):
        return self.__send_request__(endpoint='data-syndication/v1/feeds', method='GET', headers=self.headers)

    def get_channels(self):
        return self.__send_request__(endpoint='data-syndication/v1/channels', method='GET', headers=self.headers)

    def get_channel_status(self, channel_id = None):
        return self.__send_request__(endpoint='data-syndication/v1/channels/{channel_id}'.format(channel_id=channel_id),
                                     method='GET',
                                     headers=self.headers)

    def get_channel_deliveries(self, channel_id = None):
        return self.__send_request__(endpoint='data-syndication/v1/channels/{channel_id}/deliveries'.format(channel_id=channel_id),
                                     method='GET', headers=self.headers)

    def get_channel_downloads(self, delivery_id = None, destination_path = None):
        response = self.__send_request__(endpoint='data-syndication/v1/downloads/{delivery_id}'.format(delivery_id=delivery_id)
                                         , headers=self.headers)


        wd = os.getcwd()
        path = destination_path + '/' + dt.datetime.strftime(dt.datetime.today(), '%Y-%m-%d') + '.tar.gz'

        if response.status_code == 200:
            with open(path, 'wb') as i:
                i.write(response.content)
        i.close()

        temp = tarfile.open(name=path, mode='r:gz')
        os.chdir(destination_path)
        temp.extractall()
        temp.close()
        os.chdir(wd)


# Personnel Sub Class
class Personnel(Healtheintent):
    def get_personnel(self):
        return self.__send_request__(endpoint='personnel/v1/personnel', method='GET'
                                     , headers=self.headers_op)

    def create_personnel(self,data=None,json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel'.format(data=data,json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)

    def get_personnel_status(self, personnel_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_personnel(self, personnel_id=None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id, data=data, json=json)
                                     , method='PUT', headers=self.headers_op, data=data, json=json)

    def delete_personnel(self, personnel_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel/{personnel_id}'.format(personnel_id=personnel_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)

    
# Personnel Group Object
    def get_personnel_groups(self):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups'
                                     , method='GET'
                                     , headers=self.headers_op)

    def create_personnel_group(self, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups'.format(data=data,json=json)
                                     , method='POST'
                                     , headers=self.headers_op)

    def get_personnel_group_status(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_personnel_group(self, personnel_group_id = None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id, data=data,json=json)
            , method='PUT'
            , headers=self.headers_op
            , data=data
            , json=json)

    def delete_personnel_group(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}'.format(personnel_group_id=personnel_group_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)

    def get_personnel_group_members(self, personnel_group_id = None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}/members'.format(personnel_group_id=personnel_group_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    # Testing Required
    def add_personnel_group_members(self, personnel_group_id = None, personnel_group_member_id = None, data=None, json=None):
        return self.__send_request__(endpoint='personnel/v1/personnel-groups/{personnel_group_id}/members/{personnel_group_member_id}'
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)

    def delete_personnel_group_members(self, personnel_group_id = None, personnel_group_member_id = None):
        return self.__send_request__(endpoint='personnel/v1/{personnel_group_id}/members/{personnel_group_member_id}'
                                     , method='DELETE'
                                     , headers=self.headers_op)


# Organization Sub Class Commands
class Organization(Healtheintent):
    
    def get_organization(self,params=None):
        return self.__send_request__(endpoint='organization/v1/organizations'.format(params=params)
                                     , method='GET'
                                     , headers=self.headers_op)
    
    def create_organization(self,data=None,json=None):
        return self.__send_request__(endpoint='organization/v1/organizations'.format(data=data, json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)
    
    def delete_organization(self, organization_id = None):
        return self.__send_request__(endpoint='organization/v1/organizations/{organization_id}'.format(organization_id=organization_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)
    
    def update_organization(self, organization_id = None, data=None, json=None):
        return self.__send_request__(endpoint='organization/v1/organizations/{organization_id}'.format(organization_id=organization_id,data=data,json=json)
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)
    
    
    # Location Object Commands
    def get_locations(self):
        return self.__send_request__(endpoint='organization/v1/locations'
                                     , method='GET'
                                     , headers=self.headers_op)

    def create_location(self,data=None,json=None):
        return self.__send_request__(endpoint='organization/v1/locations'.format(data=data, json=json)
                                     , method='POST'
                                     , headers=self.headers_op
                                     , data=data
                                     , json=json)
    
    def get_location_status(self, location_id = None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id)
                                     , method='GET'
                                     , headers=self.headers_op)

    def update_location(self, location_id = None, data=None, json= None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id,data=data,json=json)
                                     , method='PUT'
                                     , headers=self.headers_op
                                    , data=data
                                    , json=json)

    def delete_location(self, location_id = None):
        return self.__send_request__(endpoint='organization/v1/locations/{location_id}'.format(location_id=location_id)
                                     , method='DELETE'
                                     , headers=self.headers_op)